<a href="https://colab.research.google.com/github/bluemunchkin/CSE144_SignLanguageTranslator/blob/main/CSE144G11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Loading Labels Into List

In [ ]:
def load_gloss_words(filepath):
  bag_of_words = []
  with open(filepath) as f:
    for line in f.readlines():
      tab_ind = line.find('\t')
      new_line_ind = line.find('\n')

      word = line[tab_ind + 1: new_line_ind]
      #print(word)
      bag_of_words.append(word)

  return bag_of_words

In [ ]:
class_list_file_path = "/content/drive/MyDrive/CSE144StudyGroup/ASLWordModel/ASL_data/wlasl_class_list.txt"
gloss_words = load_gloss_words(class_list_file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CSE144StudyGroup/ASLWordModel/ASL_data/wlasl_class_list.txt'

Loading Missing Video ID's (some of the videos in the json are missing)

In [ ]:
def load_missing_videos_ids(filepath):
  missing_vid_ids = []
  with open(filepath) as f:
    for line in f.readlines():
      missing_vid_ids.append(int(line[:-1]))

  return missing_vid_ids

In [ ]:
missing_video_ids= "/content/drive/MyDrive/CSE144StudyGroup/old_data/missing.txt"
missing_vid_ids= load_missing_videos_ids(missing_video_ids)

In [ ]:
len(missing_vid_ids)

9103

Loading In JSON

In [ ]:
import json
json_file_path = "/content/drive/MyDrive/CSE144StudyGroup/old_data/WLASL_v0.3.json"
f = open(json_file_path)
data = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CSE144StudyGroup/old_data/WLASL_v0.3.json'

In [ ]:
def load_json_to_dict(data):
  gloss_word_to_inst_dict = {}
  for gloss_words_info in data:
    vid_ids = []
    split = []
    for instance_struct in gloss_words_info['instances']:
      vid_id = instance_struct['video_id']
      vid_ids.append(int(vid_id))
      split.append(instance_struct['split'])
    if gloss_words_info['gloss'] not in gloss_word_to_inst_dict.keys():
      gloss_word_to_inst_dict[gloss_words_info['gloss']] = {"video_ids": vid_ids, "splits": split}
  return gloss_word_to_inst_dict

In [ ]:
def remove_missing_video_ids(instances_dict,missing_vid_ids):
  leng=0
  leng2=0
  for gloss_word in instances_dict.keys():
    vid_ids = instances_dict[gloss_word]['video_ids']

    for vid_id in vid_ids:

      if vid_id in missing_vid_ids:
        indexToUpdate = instances_dict[gloss_word]["video_ids"].index(vid_id)
        instances_dict[gloss_word]["video_ids"].remove(vid_id)
        leng += len(instances_dict[gloss_word]["video_ids"])
        instances_dict[gloss_word]["splits"].pop(indexToUpdate)
        leng2 += len(instances_dict[gloss_word]["splits"])
  print(leng,leng2)

  return instances_dict

In [ ]:
gloss_word_and_inst_dict = load_json_to_dict(data)

In [ ]:
gloss_word_and_inst_dict

In [ ]:
data_without_missing = remove_missing_video_ids(gloss_word_and_inst_dict, missing_vid_ids)

62277 62277


In [ ]:
sp = 0
gw = 0
for g in gloss_word_and_inst_dict:
  s = len(gloss_word_and_inst_dict[g]['splits'])
  sp+=s
  g = len(gloss_word_and_inst_dict[g]['video_ids'])
  gw +=g

print(sp, gw)

21083 21083


In [ ]:
def identify_differences_in_altered_vs_orig(gloss_word_and_inst_dict_orig, new_id_dict):
  orig_num_vids=0
  for gloss in gloss_word_and_inst_dict_orig.keys():
    orig_num_vids += len(gloss_word_and_inst_dict_orig[gloss]['splits'])

  after_remove_missing = 0

  for gloss in new_id_dict.keys():
    after_remove_missing += len(new_id_dict[gloss]['splits'])

  return orig_num_vids, after_remove_missing,  after_remove_missing - orig_num_vids
orig_num_vids,after_remove_missing, difference = identify_differences_in_altered_vs_orig(gloss_word_and_inst_dict, data_without_missing)

print(orig_num_vids,after_remove_missing, difference)


21083 14328 -6755


Convert the videos to Frames

In [ ]:
import cv2, os
def conv_video_to_frame(video_path, out_dir):
  #if frames saved don't re-parse them
  if os.path.exists(out_dir):
    return
  print("Entered video_to_frame")
  video_capture = cv2.VideoCapture(video_path)
  print("captured")
  os.makedirs(out_dir, exist_ok=True)

  success=True
  frame_count=0
  while success:
    success,frame = video_capture.read()
    frame_count += 1

    if frame is not None:
      frame_path = os.path.join(out_dir, f"frame_{frame_count}.jpg")
      cv2.imwrite(frame_path, frame)

    if cv2.waitKey(20) and 0xFF == ord('q'):
      break
  print("Finishing video_to_frame")
  video_capture.release()
  cv2.destroyAllWindows()

In [ ]:
#base dir paths
dataset_dir = "/content/drive/MyDrive/CSE 144 Study Group/Data/"
video_base_dir="/content/drive/MyDrive/CSE 144 Study Group/Data/videos"

#make a train folder
os.makedirs(os.path.join(dataset_dir, "Train"), exist_ok=True)
train_dir="/content/drive/MyDrive/CSE 144 Study Group/Data/Train"

#make a test folder
os.makedirs(os.path.join(dataset_dir, "Test"), exist_ok=True)
test_dir="/content/drive/MyDrive/CSE 144 Study Group/Data/Test"

def extract_frames(video_base_dir, train_dir, test_dir, data_dict):
  '''
    - extract frames for each video.
    - loop throuhg each gloss word and get all the videos associated with that word
    - extract frames from each video and store it in the folder associated with the video in the folder for that word
  '''
  for gloss_word in data_dict.keys():

    split_list = data_dict[gloss_word]["splits"]

    list_of_video_ids = data_dict[gloss_word]["video_ids"]

    for index,vid_id in enumerate(list_of_video_ids):
      os.path.join(video_base_dir, str(vid_id) + ".mp4")
      video_file_path = f"{video_base_dir}/{str(vid_id)}.mp4"
      if split_list[index] == 'train':
        os.makedirs(os.path.join(train_dir, gloss_word, str(vid_id)), exist_ok=True);
        print("Made folder for ", vid_id)
        target_file_path = f"{train_dir}/{gloss_word}/{str(vid_id)}"
        conv_video_to_frame(video_file_path, target_file_path)
      elif split_list[index] == 'test':
        os.makedirs(os.path.join(test_dir, gloss_word, str(vid_id)), exist_ok=True);
        target_file_path = f"{test_dir}/{gloss_word}/{str(vid_id)}"
        conv_video_to_frame(video_file_path, target_file_path)

'''In videos add
bad resolution videos,
angled frames,
zoomed in frames
zoomed out frames


In [ ]:
extract_frames(video_base_dir, train_dir, test_dir, data_without_missing)

In [ ]:
import shutil
import os
def sort_videos(video_base_dir, train_dir, test_dir, data_dict):
  '''
    - extract frames for each video.
    - loop throuhg each gloss word and get all the videos associated with that word
    - extract frames from each video and store it in the folder associated with the video in the folder for that word
  '''
  for gloss_word in data_dict.keys():

    split_list = data_dict[gloss_word]["splits"]

    list_of_video_ids = data_dict[gloss_word]["video_ids"]
    for index,vid_id in enumerate(list_of_video_ids):
      video_file_path = f"{video_base_dir}/{str(vid_id)}.mp4"
      if not os.path.exists(video_file_path) or os.path.exists(train_dir + "/" + str(vid_id) + ".mp4") or os.path.exists(test_dir + "/" + str(vid_id) + ".mp4"):
        continue
      if split_list[index] == 'train'  or split_list[index] == 'val':
        shutil.copy(video_file_path,train_dir)
      elif split_list[index] == 'test':
        shutil.copy(video_file_path,test_dir)

In [ ]:
len(os.listdir("/content/drive/MyDrive/CSE144StudyGroup/old_data/train_videos"))

8009

In [ ]:
train_p="/content/drive/MyDrive/CSE144StudyGroup/old_data/train_videos"
test_p="/content/drive/MyDrive/CSE144StudyGroup/old_data/test_videos"
sort_videos("/content/drive/MyDrive/CSE144StudyGroup/old_data/videos", train_p, test_p,data_without_missing)

In [ ]:
import pandas as pd
entry = pd.DataFrame.from_dict(data_without_missing)

In [ ]:


#func: put each video_id on its own line





print(l_o)
data_dict

In [ ]:
entry=entry.drop('splits',axis=0).transpose().reset_index().rename(columns={"index": "label"})

In [ ]:
type(entry)

Model to use

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_class_idx])

In [ ]:
def Video_Dataset(Dataset):
  def __init__(self,df):
    self.df=df
  def __getitem__(self,i):
    return self.df[i]['video_id'], self.df[i]['label']


In [ ]:
!pip install av

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 17.7 MB/s eta 0:00:00


In [ ]:
import av
import torch
import numpy as np

from transformers import AutoImageProcessor, VideoMAEForVideoClassification
from huggingface_hub import hf_hub_download

np.random.seed(0)


def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices


# video clip consists of 300 frames (10 seconds at 30 FPS)
file_path = hf_hub_download(
    repo_id="nielsr/video-demo", filename="eating_spaghetti.mp4", repo_type="dataset"
)
container = av.open(file_path)

# sample 16 frames
indices = sample_frame_indices(clip_len=16, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
video = read_video_pyav(container, indices)

image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")
model = VideoMAEForVideoClassification.from_pretrained("MCG-NJU/videomae-base-finetuned-kinetics")

inputs = image_processor(list(video), return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# model predicts one of the 400 Kinetics-400 classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label])


eating_spaghetti.mp4:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


config.json:   0%|          | 0.00/22.9k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(value)


eating spaghetti
